# S.M.A.R.T QUESTIONS

1. Analisis Kunjungan Pasien
- Apakah ada pola tertentu dalam frekuensi kunjungan pasien berdasarkan tanggal pemeriksaan (tanggal_periksa)?
- Berapa rata-rata jumlah kunjungan per pasien dalam satu bulan? Apakah ada pasien yang sering berulang kali datang?
2. Analisis Berdasarkan Klinik dan Dokter
- Klinik atau poli mana yang paling sering dikunjungi pasien? Adakah hubungan antara jenis kunjungan (jenis_kunjungan) dengan klinik tertentu?
- Dokter mana yang memiliki jumlah pasien terbanyak? Adakah hubungan antara jumlah kunjungan dengan hari tertentu?
3. Analisis Status BPJS dan Keberhasilan Pengiriman
- Berapa persentase keberhasilan dan kegagalan status pengiriman data (status_kirim)? Apakah ada tren tertentu dalam data yang gagal terkirim?
- Adakah pola tertentu dalam kegagalan pengiriman berdasarkan jenis kunjungan atau poli tertentu?
4. Analisis Berdasarkan Jenis Kunjungan dan BPJS
- Jenis kunjungan (jenis_kunjungan) mana yang paling dominan dalam dataset? Apakah pasien dengan kunjungan tertentu lebih sering mengalami kegagalan pengiriman?
- Apakah pasien dengan nomor kartu BPJS yang sama sering berkunjung ke berbagai klinik atau hanya ke klinik tertentu?
5. Analisis Perbandingan Kinerja Berdasarkan Tanggal
- Apakah ada peningkatan atau penurunan jumlah pasien pada tanggal tertentu (misalnya, akhir pekan vs hari kerja)?
- Adakah hari tertentu dengan volume pasien tinggi yang menyebabkan peningkatan kegagalan pengiriman?
6. Identifikasi Pasien dengan Kunjungan Berulang dan Nomor BPJS
- Berapa banyak pasien yang melakukan lebih dari satu kunjungan dalam periode yang sama? Apakah mereka lebih rentan terhadap kegagalan pengiriman status?
- Apakah pasien dengan nomor BPJS tertentu lebih sering mengalami masalah status kirim?

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib as mpl
import matplotlib.patches as mpatches 
from matplotlib.colors import LinearSegmentedColormap
import warnings 
import missingno as ms


warnings.filterwarnings('ignore')
sns.set(style='whitegrid')
plt.style.use('Solarize_Light2')


# Data Preparation

## Gathering Data

In [2]:
df_bpjs = pd.read_csv('./data/juli 2024.csv')

In [3]:
pd.DataFrame(df_bpjs)

no_rawat tanggal_periksa                nm_poli  \
0     2024/07/01/000004      31/07/2024            KLINIK MATA   
1     2024/07/01/000004      15/07/2024            KLINIK MATA   
2     2024/07/01/000004      19/07/2024            KLINIK MATA   
3     2024/07/01/000004       12/7/2024            KLINIK MATA   
4     2024/07/01/000004      15/07/2024            KLINIK MATA   
...                 ...             ...                    ...   
1354  2024/07/04/000121      15/07/2024  KLINIK PENYAKIT DALAM   
1355  2024/07/04/000121      22/07/2024  KLINIK PENYAKIT DALAM   
1356  2024/07/04/000121        4/7/2024  KLINIK PENYAKIT DALAM   
1357  2024/07/04/000123      25/07/2024            KLINIK MATA   
1358  2024/07/04/000123        4/7/2024                    NaN   

                                   nm_dokter     nm_pasien  no_rkm_medis  \
0                      dr. H. Rezekian, Sp.M  MARNI Hj. Ny       97522.0   
1                      dr. H. Rezekian, Sp.M  MARNI Hj. Ny       97522.0   
2                      dr. H. Rezekian, Sp.M  MARNI Hj. Ny       97522.0   
3                      dr. H. Rezekian, Sp.M  MARNI Hj. Ny       97522.0   
4                      dr. H. Rezekian, Sp.M  MARNI Hj. Ny       97522.0   
...                                      ...           ...           ...   
1354  dr. CHARISMA BIMARA COZY, M.Kes, Sp.PD    RIDUAN, Tn      224148.0   
1355  dr. CHARISMA BIMARA COZY, M.Kes, Sp.PD    RIDUAN, Tn      224148.0   
1356  dr. CHARISMA BIMARA COZY, M.Kes, Sp.PD    RIDUAN, Tn      224148.0   
1357                   dr. H. Rezekian, Sp.M  H.JONI KARLI      194620.0   
1358                                     NaN           NaN           NaN   

           png_jawab nomor_kartu      nomor_referensi  \
0     BPJS Kesehatan  1080583615  170603010624Y000093   
1     BPJS Kesehatan  1080583615  170603010724Y000496   
2     BPJS Kesehatan  1080583615  170603010624Y000093   
3     BPJS Kesehatan  1080583615  1706R0060624K003795   
4     BPJS Kesehatan  1080583615  170603010724Y000496   
...              ...         ...                  ...   
1354  BPJS Kesehatan  2919164264  170604020624P001102   
1355  BPJS Kesehatan  2919164264  170604020624P001102   
1356  BPJS Kesehatan  2919164264  170604020624P001102   
1357  BPJS Kesehatan  3304753233  170607010424P001248   
1358             NaN         NaN                  NaN   

                           kodebooking  jenis_kunjungan status_kirim  
0         1706R00620240731000020MAT004              3.0        Sudah  
1     1706R00620240715093247JIW005MJKN              1.0        Gagal  
2         1706R00620240719000016MAT001              3.0        Sudah  
3     1706R00620240708034048KLT001MJKN              3.0        Gagal  
4         1706R00620240715000190JIW005              3.0        Sudah  
...                                ...              ...          ...  
1354      1706R00620240715000074INT003              3.0        Sudah  
1355      1706R00620240722000120INT014              3.0        Sudah  
1356      1706R00620240704000121INT018              3.0        Sudah  
1357      1706R00620240725000061MAT019              3.0        Sudah  
1358                               NaN              NaN          NaN  

[1359 rows x 12 columns]

In [4]:
print(f"Unique of BPJS in the datasets : {len(df_bpjs.no_rawat.unique())}")

Unique of BPJS in the datasets : 640


## assesing data

In [5]:
df_bpjs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1359 entries, 0 to 1358
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   no_rawat         1359 non-null   object 
 1   tanggal_periksa  1359 non-null   object 
 2   nm_poli          1358 non-null   object 
 3   nm_dokter        1358 non-null   object 
 4   nm_pasien        1358 non-null   object 
 5   no_rkm_medis     1358 non-null   float64
 6   png_jawab        1358 non-null   object 
 7   nomor_kartu      1305 non-null   object 
 8   nomor_referensi  1358 non-null   object 
 9   kodebooking      1358 non-null   object 
 10  jenis_kunjungan  1358 non-null   float64
 11  status_kirim     1358 non-null   object 
dtypes: float64(2), object(10)
memory usage: 127.5+ KB


- no_rawat (object): Nomor perawatan pasien, terdiri dari 1359 nilai yang semuanya terisi (tidak ada yang kosong).
- tanggal_periksa (object): Tanggal pemeriksaan pasien, terdiri dari 1359 nilai yang semuanya terisi.
- nm_poli (object): Nama poli (klinik) tempat pasien diperiksa, terdiri dari 1358 nilai yang terisi, artinya ada 1 nilai yang kosong.
- nm_dokter (object): Nama dokter yang menangani pasien, terdiri dari 1358 nilai yang terisi, ada 1 nilai yang kosong.
- nm_pasien (object): Nama pasien, terdiri dari 1358 nilai yang terisi, ada 1 nilai yang kosong.
- no_rkm_medis (float64): Nomor rekam medis pasien, terdiri dari 1358 nilai yang terisi, ada 1 nilai yang kosong.
- png_jawab (object): Penanggung jawab pembayaran, terdiri dari 1358 nilai yang terisi, ada 1 nilai yang kosong.
- nomor_kartu (object): Nomor kartu BPJS pasien, hanya 1305 nilai yang terisi, artinya ada 54 nilai yang kosong.
- nomor_referensi (object): Nomor referensi dari sistem antrian, terdiri dari 1358 nilai yang terisi, ada 1 nilai yang kosong.
- kodebooking (object): Kode booking pasien, terdiri dari 1358 nilai yang terisi, ada 1 nilai yang kosong.
- jenis_kunjungan (float64): Jenis kunjungan (misalnya rawat jalan, kontrol, dll.), terdiri dari 1358 nilai yang terisi, ada 1 nilai yang kosong.
- status_kirim (object): Status pengiriman data, terdiri dari 1358 nilai yang terisi, ada 1 nilai yang kosong.

In [6]:
print("DataFrame Tourism Information : ")
print(df_bpjs.isna().sum())

DataFrame Tourism Information : 
no_rawat            0
tanggal_periksa     0
nm_poli             1
nm_dokter           1
nm_pasien           1
no_rkm_medis        1
png_jawab           1
nomor_kartu        54
nomor_referensi     1
kodebooking         1
jenis_kunjungan     1
status_kirim        1
dtype: int64


In [7]:
ms.matrix(df_bpjs)

<Axes: >

In [8]:
df_bpjs.sample(10)

no_rawat tanggal_periksa                nm_poli  \
318   2024/07/01/000178        8/7/2024  KLINIK PENYAKIT DALAM   
276   2024/07/01/000157      19/07/2024           KLINIK BEDAH   
670   2024/07/02/000129        2/7/2024             KLINIK THT   
9     2024/07/01/000006        1/7/2024            KLINIK MATA   
1064  2024/07/03/000236        9/7/2024            KLINIK GIGI   
746   2024/07/02/000178        2/7/2024            KLINIK GIGI   
1261  2024/07/04/000072      18/07/2024           KLINIK SARAF   
792   2024/07/03/000016      22/07/2024            KLINIK MATA   
363   2024/07/01/000205      22/07/2024            KLINIK MATA   
744   2024/07/02/000178        2/7/2024            KLINIK GIGI   

                                   nm_dokter                      nm_pasien  \
318   dr. CHARISMA BIMARA COZY, M.Kes, Sp.PD                    RUSMINI, Ny   
276                     dr. H. Mujiran, Sp.B     DHONA LIDYA FEBRIYANTI, Ny   
670             dr. Anita Nuraini, Sp.THT-KL                     NORMANSYAH   
9                      dr. H. Rezekian, Sp.M          HUMAIRA ZAHRA NUR, An   
1064    drg. Hj. May Salamah Emma Ganefi, MM                        SAPUWAN   
746     drg. Hj. May Salamah Emma Ganefi, MM  NADIAH SALMA AMIRAH AL FAWWAZ   
1261                dr. Yenni Yustini, Sp. S                  AMINUDDIN, Tn   
792                    dr. H. Rezekian, Sp.M             ASHWIN PARDEDE, Tn   
363                    dr. H. Rezekian, Sp.M                    LAMSIAH, Ny   
744     drg. Hj. May Salamah Emma Ganefi, MM  NADIAH SALMA AMIRAH AL FAWWAZ   

      no_rkm_medis       png_jawab nomor_kartu            nomor_referensi  \
318       223716.0  BPJS Kesehatan  3225971643        170604020724P000249   
276       223543.0  BPJS Kesehatan  2752293543        170601010724P000877   
670       186430.0  BPJS Kesehatan  2886875897        170601020724P000001   
9         224024.0            UMUM         NaN  1706R00620240701000006004   
1064       61443.0  BPJS Kesehatan   124157057        170601010524P001039   
746       224082.0            UMUM         NaN  1706R00620240702000178004   
1261         880.0  BPJS Kesehatan   124282293        170601010624Y001421   
792       151562.0  BPJS Kesehatan  2446014003        0285U0230524P000086   
363       221400.0  BPJS Kesehatan  3583585315        170601010624Y000133   
744       224082.0            UMUM         NaN  1706R00620240702000178004   

                       kodebooking  jenis_kunjungan status_kirim  
318   1706R00620240708000178INT020              3.0        Gagal  
276   1706R00620240719000010JIW001              3.0        Gagal  
670   1706R00620240702000129THT008              3.0        Sudah  
9     1706R00620240701000006MAT004              3.0        Sudah  
1064  1706R00620240709000172GIG005              3.0        Sudah  
746   1706R00620240702000178GIG004              3.0        Sudah  
1261  1706R00620240718000066SAR008              3.0        Gagal  
792   1706R00620240722000010MAT004              3.0        Sudah  
363   1706R00620240722000312MAT061              3.0        Gagal  
744   1706R00620240702000178GIG004              3.0        Sudah

In [9]:
print("summary data bpjs pasien duplicated : ", df_bpjs.duplicated().sum())

summary data bpjs pasien duplicated :  187


terdapat data pasien dari bpjs terduplikat 187 

In [10]:
df_bpjs.describe(include='all').style.background_gradient(cmap="RdPu")

(-) Descriptive Analysis of tourism information

Dataset tourism information

- summary data entries : 1359
- Q1 and Q3 : 3.0 and 3.0
- standard deviation jenis kunjungan : 0.187
- Rata-rata no_rkm_medis : 174983
- mean Rating : 2.98
<!-- - mean Time Minutes : 82 minutes
- mean Latitude : -7.095438
- mean Longtitude : 109.160142 -->


## Cleaning data 

Metode Imputation

In [11]:
df_bpjs[df_bpjs.nomor_kartu.isna()]

no_rawat tanggal_periksa                nm_poli  \
9     2024/07/01/000006        1/7/2024            KLINIK MATA   
10    2024/07/01/000006        1/7/2024            KLINIK MATA   
15    2024/07/01/000011        1/7/2024           KLINIK BEDAH   
16    2024/07/01/000011        1/7/2024           KLINIK BEDAH   
253   2024/07/01/000143      30/07/2024            KLINIK ANAK   
283   2024/07/01/000165        1/7/2024           KLINIK BEDAH   
284   2024/07/01/000165        1/7/2024           KLINIK BEDAH   
342   2024/07/01/000190        1/7/2024  KLINIK PENYAKIT DALAM   
343   2024/07/01/000190        1/7/2024  KLINIK PENYAKIT DALAM   
392   2024/07/01/000228      15/07/2024            KLINIK ANAK   
393   2024/07/01/000228      15/07/2024            KLINIK ANAK   
394   2024/07/01/000228      15/07/2024            KLINIK ANAK   
395   2024/07/01/000228      15/07/2024            KLINIK ANAK   
396   2024/07/01/000228      15/07/2024            KLINIK ANAK   
397   2024/07/01/000228      15/07/2024            KLINIK ANAK   
447   2024/07/01/000251        1/7/2024  KLINIK PENYAKIT DALAM   
448   2024/07/01/000251        1/7/2024  KLINIK PENYAKIT DALAM   
449   2024/07/01/000251        1/7/2024  KLINIK PENYAKIT DALAM   
450   2024/07/01/000251        1/7/2024  KLINIK PENYAKIT DALAM   
451   2024/07/01/000251        1/7/2024  KLINIK PENYAKIT DALAM   
461   2024/07/01/000257        1/7/2024            KLINIK PARU   
464   2024/07/01/000257        1/7/2024            KLINIK PARU   
472   2024/07/01/000260        1/7/2024   KLINIK OBSTETRI/GYN.   
473   2024/07/01/000260        1/7/2024   KLINIK OBSTETRI/GYN.   
474   2024/07/01/000260        1/7/2024   KLINIK OBSTETRI/GYN.   
475   2024/07/01/000260        1/7/2024   KLINIK OBSTETRI/GYN.   
508   2024/07/01/000268        1/7/2024   KLINIK OBSTETRI/GYN.   
509   2024/07/01/000268        1/7/2024   KLINIK OBSTETRI/GYN.   
510   2024/07/01/000268        1/7/2024   KLINIK OBSTETRI/GYN.   
511   2024/07/01/000268        1/7/2024   KLINIK OBSTETRI/GYN.   
513   2024/07/01/000268        1/7/2024   KLINIK OBSTETRI/GYN.   
514   2024/07/01/000268        1/7/2024   KLINIK OBSTETRI/GYN.   
524   2024/07/01/000274        1/7/2024           KLINIK BEDAH   
525   2024/07/01/000274        1/7/2024           KLINIK BEDAH   
533   2024/07/01/000278        1/7/2024  KLINIK PENYAKIT DALAM   
534   2024/07/01/000278        1/7/2024  KLINIK PENYAKIT DALAM   
642   2024/07/02/000099        2/7/2024           KLINIK BEDAH   
643   2024/07/02/000099        2/7/2024           KLINIK BEDAH   
742   2024/07/02/000178        2/7/2024            KLINIK GIGI   
743   2024/07/02/000178        2/7/2024            KLINIK GIGI   
744   2024/07/02/000178        2/7/2024            KLINIK GIGI   
745   2024/07/02/000178        2/7/2024            KLINIK GIGI   
746   2024/07/02/000178        2/7/2024            KLINIK GIGI   
747   2024/07/02/000178        2/7/2024            KLINIK GIGI   
748   2024/07/02/000178        2/7/2024            KLINIK GIGI   
749   2024/07/02/000178        2/7/2024            KLINIK GIGI   
750   2024/07/02/000178        2/7/2024            KLINIK GIGI   
1161  2024/07/03/000309        3/7/2024  KLINIK PENYAKIT DALAM   
1163  2024/07/03/000309        3/7/2024  KLINIK PENYAKIT DALAM   
1164  2024/07/03/000309        3/7/2024  KLINIK PENYAKIT DALAM   
1228  2024/07/04/000039        4/7/2024             KLINIK THT   
1229  2024/07/04/000039        4/7/2024             KLINIK THT   
1230  2024/07/04/000039        4/7/2024             KLINIK THT   
1358  2024/07/04/000123        4/7/2024                    NaN   

                                   nm_dokter                      nm_pasien  \
9                      dr. H. Rezekian, Sp.M          HUMAIRA ZAHRA NUR, An   
10                     dr. H. Rezekian, Sp.M          HUMAIRA ZAHRA NUR, An   
15                      dr. H. Mujiran, Sp.B                  NOR ASIFA, An   
16                      dr. H. Mujiran, Sp.B                  NOR AS

In [12]:
pd.DataFrame(df_bpjs.nomor_kartu.value_counts())

count
nomor_kartu       
2090136969      18
3396544751      18
124144391       15
2457752624      12
0               12
...            ...
171893158        1
977782779        1
3396495251       1
1897843667       1
3304753233       1

[598 rows x 1 columns]

In [13]:
print("summary data bpjs pasien duplicated : ", df_bpjs.duplicated().sum())

summary data bpjs pasien duplicated :  187


In [14]:
if df_bpjs.duplicated().sum() > 0:
    df_bpjs.drop_duplicates(inplace=True)
    df_bpjs.reset_index(inplace=True, drop=True)

print("Jumlah Data yang duplicate", df_bpjs.duplicated().sum())

Jumlah Data yang duplicate 0


In [15]:
df_bpjs[['nm_poli', 'nm_dokter', 'nm_pasien', 'no_rkm_medis', 'png_jawab', 
         'nomor_kartu', 'nomor_referensi', 'kodebooking', 'jenis_kunjungan', 'status_kirim']] = df_bpjs[['nm_poli', 'nm_dokter', 'nm_pasien', 'no_rkm_medis', 'png_jawab', 
         'nomor_kartu', 'nomor_referensi', 'kodebooking', 'jenis_kunjungan', 'status_kirim']].fillna('Unknown')

# Menampilkan jumlah NaN per kolom setelah diisi
print("\nJumlah NaN setelah pengisian:")
print(df_bpjs.isna().sum())


Jumlah NaN setelah pengisian:
no_rawat           0
tanggal_periksa    0
nm_poli            0
nm_dokter          0
nm_pasien          0
no_rkm_medis       0
png_jawab          0
nomor_kartu        0
nomor_referensi    0
kodebooking        0
jenis_kunjungan    0
status_kirim       0
dtype: int64


In [16]:
df_bpjs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1172 entries, 0 to 1171
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   no_rawat         1172 non-null   object
 1   tanggal_periksa  1172 non-null   object
 2   nm_poli          1172 non-null   object
 3   nm_dokter        1172 non-null   object
 4   nm_pasien        1172 non-null   object
 5   no_rkm_medis     1172 non-null   object
 6   png_jawab        1172 non-null   object
 7   nomor_kartu      1172 non-null   object
 8   nomor_referensi  1172 non-null   object
 9   kodebooking      1172 non-null   object
 10  jenis_kunjungan  1172 non-null   object
 11  status_kirim     1172 non-null   object
dtypes: object(12)
memory usage: 110.0+ KB


In [17]:
# df_bpjs[df_bpjs.nomor_kartu.isna()]
df_bpjs.describe(include='all').T.style.background_gradient(cmap='RdPu')

In [18]:
# plt.figure(figsize=(20,20))
# sns.heatmap(df_bpjs.corr() , annot=True, cmap='magma', linecolor='white', linewidth=3)
# plt.title('Correlation Heatmap')
# plt.show()

# df_bpjs.hist(bins=50, figsize=(40, 25), color='#0661D1', edgecolor='black')
# plt.show()

# Exploratory Data Analysis (EDA)

In [19]:
df_bpjs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1172 entries, 0 to 1171
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   no_rawat         1172 non-null   object
 1   tanggal_periksa  1172 non-null   object
 2   nm_poli          1172 non-null   object
 3   nm_dokter        1172 non-null   object
 4   nm_pasien        1172 non-null   object
 5   no_rkm_medis     1172 non-null   object
 6   png_jawab        1172 non-null   object
 7   nomor_kartu      1172 non-null   object
 8   nomor_referensi  1172 non-null   object
 9   kodebooking      1172 non-null   object
 10  jenis_kunjungan  1172 non-null   object
 11  status_kirim     1172 non-null   object
dtypes: object(12)
memory usage: 110.0+ KB


In [20]:
df_bpjs.status_kirim.unique()

array(['Sudah', 'Gagal', 'Belum', 'Unknown'], dtype=object)

In [21]:
df_bpjs.jenis_kunjungan.unique()

array([3.0, 1.0, 'Unknown'], dtype=object)

In [22]:
df_bpjs.nm_poli.unique()

array(['KLINIK MATA', 'KLINIK PENYAKIT DALAM', 'KLINIK BEDAH',
       'KLINIK OBSTETRI/GYN.', 'KLINIK PARU', 'KLINIK JIWA',
       'KLINIK GIGI', 'KLINIK KULIT & KELAMIN', 'KLINIK SARAF',
       'KLINIK ANAK', 'KLINIK THT', 'KLINIK UMUM / MCU', 'Unknown'],
      dtype=object)

In [23]:
numeric_cols_information = ['no_rawat', 'tanggal_periksa']

# Set up subplots
fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(10, 8))
fig.suptitle('Distribusi Variabel Numerik Tourism Information', y=1.02, fontsize=16)

# Flatten the axes for easy iteration
axes = axes.flatten()

# Loop through numeric columns and create histograms
for i, col in enumerate(numeric_cols_information):
    sns.histplot(df_bpjs[col], kde=True, ax=axes[i], bins=30)
    axes[i].set_title(f'Distribusi {col}')

# Adjust layout to prevent overlap
plt.tight_layout()
plt.show()